In [1]:
import nltk

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

import json

import pickle

 

import numpy as np

from keras.models import Sequential

from keras.layers import Dense, Activation, Dropout

from keras.optimizers import SGD

import random



In [2]:
 

words=[]

classes = []

documents = []

ignore_words = ['?', '!']

data_file = open("intents.json").read()

intents = json.loads(data_file)

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [4]:
for intent in intents['intents']:

    for pattern in intent['patterns']:

 

        #tokenize each word

        w = nltk.word_tokenize(pattern)

        words.extend(w)

        #add documents in the corpus

        documents.append((w, intent['tag']))

 

        # add to our classes list

        if intent['tag'] not in classes:

            classes.append(intent['tag'])

In [5]:
from nltk.corpus import wordnet as wn
# Example: Find synonyms for the word "dog"
synonyms = wn.synsets('dog')
print(synonyms[0].definition())  # Prints the definition of the first synonym for 'dog'


a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds


In [6]:
# lemmatize, lower each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

words = sorted(list(set(words)))

# sort classes

classes = sorted(list(set(classes)))

# documents = combination between patterns and intents

print (len(documents), "documents")

# classes = intents

print (len(classes), "classes", classes)

# words = all words, vocabulary

print (len(words), "unique lemmatized words", words)

 

pickle.dump(words,open('words.pkl','wb'))

pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [7]:
# create our training data

training = []

# create an empty array for our output

output_empty = [0] * len(classes)

# training set, bag of words for each sentence

for doc in documents:

    # initialize our bag of words

    bag = []

    # list of tokenized words for the pattern

    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words

    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern

    for w in words:

        bag.append(1) if w in pattern_words else bag.append(0)

# output is a '0' for each tag and '1' for current tag (for each pattern)

    output_row = list(output_empty)

    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array

import numpy as np
import random

# Assuming 'words', 'classes', 'documents', and 'lemmatizer' are defined elsewhere

training = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # Initialize output row inside the loop
    output_row = list(output_empty)

    for w in words:
        # Check if word is in the pattern_words
        bag.append(1) if w in pattern_words else bag.append(0)
        
    # Set the corresponding index of the output_row to 1
    output_row[classes.index(doc[1])] = 1

    # Append the bag and output_row as a tuple to training
    training.append((bag, output_row))

# Shuffle training data before converting to numpy array
random.shuffle(training)

# Print the shapes of each element in the training list
for item in training:
    print("Shape of bag:", np.array(item[0]).shape)
    print("Shape of output_row:", np.array(item[1]).shape)

# Separate features (X) and labels (Y) from the training data
train_x = np.array([bag for bag, _ in training])
train_y = np.array([output_row for _, output_row in training])

print("Training data created")








Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of output_row: (9,)
Shape of bag: (88,)
Shape of outpu

In [9]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons, and 3rd output layer contains the number of neurons
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model using the new Keras optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Fit the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Save the model
model.save('chatbot_model.h5')

print("Model created")


Epoch 1/200
10/10 [==============================] - 5s 14ms/step - loss: 2.2153 - accuracy: 0.0851
Epoch 2/200
10/10 [==============================] - 0s 10ms/step - loss: 2.1591 - accuracy: 0.2979
Epoch 3/200
10/10 [==============================] - 0s 11ms/step - loss: 2.1108 - accuracy: 0.1702
Epoch 4/200
10/10 [==============================] - 0s 10ms/step - loss: 2.1177 - accuracy: 0.2553
Epoch 5/200
10/10 [==============================] - 0s 11ms/step - loss: 1.8790 - accuracy: 0.4894
Epoch 6/200
10/10 [==============================] - 0s 11ms/step - loss: 1.7626 - accuracy: 0.4043
Epoch 7/200
10/10 [==============================] - 0s 12ms/step - loss: 1.6527 - accuracy: 0.4681
Epoch 8/200
10/10 [==============================] - 0s 10ms/step - loss: 1.5121 - accuracy: 0.5957
Epoch 9/200
10/10 [==============================] - 0s 11ms/step - loss: 1.5046 - accuracy: 0.5532
Epoch 10/200
10/10 [==============================] - 0s 16ms/step - loss: 1.2910 - accuracy: 0.5745

10/10 [==============================] - 0s 18ms/step - loss: 0.1712 - accuracy: 0.9149
Epoch 83/200
10/10 [==============================] - 0s 19ms/step - loss: 0.0530 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 30ms/step - loss: 0.0432 - accuracy: 0.9787
Epoch 85/200
10/10 [==============================] - 0s 22ms/step - loss: 0.0567 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 14ms/step - loss: 0.0562 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 13ms/step - loss: 0.0246 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 11ms/step - loss: 0.0744 - accuracy: 0.9787
Epoch 89/200
10/10 [==============================] - 0s 11ms/step - loss: 0.0900 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 11ms/step - loss: 0.0279 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 16ms/step - loss: 0.0581 - accuracy: 0.9787
Epo

10/10 [==============================] - 0s 7ms/step - loss: 0.0640 - accuracy: 0.9787
Epoch 164/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0264 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0278 - accuracy: 0.9787
Epoch 166/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0199 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0361 - accuracy: 0.9787
Epoch 168/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0052 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0363 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0287 - accuracy: 0.9787
Epoc

C:\Users\DEBANIK DEB\AnacondasPython\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model created


In [11]:
import nltk

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

import pickle

import numpy as np

 

from keras.models import load_model

model = load_model('chatbot_model.h5')

import json

import random

intents = json.loads(open("intents.json").read())

words = pickle.load(open('words.pkl','rb'))

classes = pickle.load(open('classes.pkl','rb'))

In [12]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array

    sentence_words = nltk.word_tokenize(sentence)

    # stem each word - create short form for word

    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):

    # tokenize the pattern

    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix

    bag = [0]*len(words) 

    for s in sentence_words:

        for i,w in enumerate(words):

            if w == s: 

                # assign 1 if current word is in the vocabulary position

                bag[i] = 1

                if show_details:

                    print ("found in bag: %s" % w)

    return(np.array(bag))

def predict_class(sentence, model):

    # filter out predictions below a threshold

    p = bow(sentence, words,show_details=False)

    res = model.predict(np.array([p]))[0]

    ERROR_THRESHOLD = 0.25

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    # sort by strength of probability

    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []

    for r in results:

        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list




In [13]:
def getResponse(ints, intents_json):

    tag = ints[0]['intent']

    list_of_intents = intents_json['intents']

    for i in list_of_intents:

        if(i['tag']== tag):

            result = random.choice(i['responses'])

            break

    return result

def chatbot_response(text):

    ints = predict_class(text, model)

    res = getResponse(ints, intents)

    return res

In [ ]:
#Creating GUI with tkinter

import tkinter

from tkinter import *

def send():

    msg = EntryBox.get("1.0",'end-1c').strip()

    EntryBox.delete("0.0",END)

    if msg != '':

        ChatLog.config(state=NORMAL)

        ChatLog.insert(END, "You: " + msg + '\n\n')

        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)

        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)

        ChatLog.yview(END)

base = Tk()

base.title("Hello")

base.geometry("400x500")

base.resizable(width=FALSE, height=FALSE)

#Create Chat window

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")

ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message

SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,

                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',

                    command= send )

#Create the box to enter message

EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")

#EntryBox.bind("<Return>", send)

#Place all components on the screen

scrollbar.place(x=376,y=6, height=386)

ChatLog.place(x=6,y=6, height=386, width=370)

EntryBox.place(x=128, y=401, height=90, width=265)

SendButton.place(x=6, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 88ms/step
